# Payoff d'une butterfly call sur SPY

Structure butterfly (long K1, short 2xK2, long K3) : payoff centré sur K2, P&L net (prime BS) et points morts.


In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
import sys
from pathlib import Path
from IPython.display import display, Markdown

_base = Path.cwd().resolve()
for extra in (_base, _base.parent, _base.parent.parent):
    candidate = extra / "notebooks" / "scripts"
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))
        break
from pricing import fetch_spy_history, view_butterfly, payoff_butterfly
plt.style.use("seaborn-v0_8-darkgrid")

close_spy = fetch_spy_history()
spot_ref = float(close_spy.iloc[-1])

k1 = spot_ref * 0.95
k2 = spot_ref
k3 = spot_ref * 1.05
view_static = view_butterfly(spot_ref, k1, k2, k3)
premium = view_static["premium"]
be_low, be_high = view_static["breakevens"]


## Évolution du sous-jacent (SPY)
Historique des clôtures sur un an et repère du spot de référence (dernier close).


In [ ]:

fig, ax = plt.subplots(figsize=(10, 4))
close_spy.plot(ax=ax, color="steelblue", label="SPY close")
ax.axhline(spot_ref, color="crimson", linestyle="--", label=f"Dernier close ≈ {spot_ref:.2f}")
ax.set_title("SPY - clôtures sur 1 an")
ax.set_xlabel("Date")
ax.set_ylabel("Prix")
ax.legend()
plt.show()


## Payoff de la butterfly call
Payoff en "tiroir" autour de K2, borné et symétrique. Visualisation du payoff, du P&L net et des points morts.


In [ ]:

    fig, ax = plt.subplots(figsize=(9, 5))
    ax.plot(view_static["s_grid"], view_static["payoff"], label="Payoff brut")
    ax.plot(view_static["s_grid"], view_static["pnl"], label="P&L net", color="darkorange")
    ax.axvline(k1, color="gray", linestyle=":", label=f"K1 = {k1:.2f}")
    ax.axvline(k2, color="gray", linestyle="--", label=f"K2 = {k2:.2f}")
    ax.axvline(k3, color="gray", linestyle=":", label=f"K3 = {k3:.2f}")
    ax.axvline(be_low, color="forestgreen", linestyle=":", label=f"Point mort bas ≈ {be_low:.2f}")
    ax.axvline(be_high, color="forestgreen", linestyle="--", label=f"Point mort haut ≈ {be_high:.2f}")
    ax.axvline(spot_ref, color="crimson", linestyle="-.", label=f"S_0 = {spot_ref:.2f}")
    ax.axhline(0, color="black", linewidth=0.8)
    ax.set_xlabel("Spot")
    ax.set_ylabel("Payoff / P&L")
    ax.set_title("Butterfly call : payoff et P&L (prime BS)")
    ax.legend(loc="upper left")
    plt.show()

    display(Markdown(f"""**Prime (BS) ≈ {premium:.4f}**

- Points morts : {be_low:.2f} / {be_high:.2f}
- Structure centrée sur K2."""))


## Payoff interactif
Curseurs pour ajuster S_0, S_T et K1/K2/K3, prime et points morts recalculés.


In [ ]:

    spot0_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_0')
    spotT_slider = widgets.FloatSlider(value=spot_ref, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='S_T')
    slider_k1 = widgets.FloatSlider(value=k1, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K1')
    slider_k2 = widgets.FloatSlider(value=k2, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K2')
    slider_k3 = widgets.FloatSlider(value=k3, min=spot_ref*0.5, max=spot_ref*1.5, step=1.0, description='K3')
    output = widgets.Output()

    def _update_payoff(change=None):
        with output:
            output.clear_output()
            s0 = spot0_slider.value
            sT = spotT_slider.value
            k1_val = slider_k1.value
            k2_val = slider_k2.value
            k3_val = slider_k3.value

            view_dyn = view_butterfly(s0, k1_val, k2_val, k3_val)
            premium_dyn = view_dyn["premium"]
            be_low, be_high = view_dyn["breakevens"]
            payoff_T = float(payoff_butterfly(sT, k1_val, k2_val, k3_val))
            pnl_T = payoff_T - premium_dyn

            fig, ax = plt.subplots(figsize=(7, 4))
            ax.plot(view_dyn["s_grid"], view_dyn["payoff"], label='Payoff brut')
            ax.plot(view_dyn["s_grid"], view_dyn["pnl"], label='P&L net', color='darkorange')
            for k, style, label in [
                (k1_val, ':', f'K1 = {k1_val:.2f}'),
                (k2_val, '--', f'K2 = {k2_val:.2f}'),
                (k3_val, ':', f'K3 = {k3_val:.2f}'),
            ]:
                ax.axvline(k, color='gray', linestyle=style, label=label)
            ax.axvline(be_low, color='forestgreen', linestyle=':', label=f'Point mort bas ≈ {be_low:.2f}')
            ax.axvline(be_high, color='forestgreen', linestyle='--', label=f'Point mort haut ≈ {be_high:.2f}')
            ax.axvline(s0, color='crimson', linestyle='-.', label=f'S_0 = {s0:.2f}')
            ax.axhline(0, color='black', linewidth=0.8)
            ax.scatter([sT], [pnl_T], color='gold', zorder=5, label=f'P&L à S_T = {pnl_T:.2f}')
            ax.set_xlabel('Spot')
            ax.set_ylabel('Payoff / P&L')
            ax.legend(loc='lower right')
            ax.set_title('Payoff et P&L de la butterfly')
            plt.show()

            display(Markdown(f"""**S_0 = {s0:.2f}**, **S_T = {sT:.2f}**

- K1 = {k1_val:.2f}
- K2 = {k2_val:.2f}
- K3 = {k3_val:.2f}
- Prix (BS) = {premium_dyn:.4f}
- Payoff à l'expiration = {payoff_T:.4f}
- P&L net = {pnl_T:.4f}
- Points morts ≈ {be_low:.2f} / {be_high:.2f}"""))

    for sl in (spot0_slider, spotT_slider, slider_k1, slider_k2, slider_k3):
        sl.observe(_update_payoff, names='value')

    _update_payoff()
    display(widgets.VBox([spot0_slider, spotT_slider, slider_k1, slider_k2, slider_k3, output]))
